In [1]:
import sys
import os
sys.path.insert(0, f'{os.getcwd()}/third_party/lanet')

import coremltools as ct
from coremltools.models.neural_network import flexible_shape_utils
from third_party.lanet.network_v1.model import PointModel as PointModel_v1
from third_party.lanet.network_v0.model import PointModel as PointModel_v0
import torch
import cv2
import matplotlib.pyplot as plt
import kornia as K
import numpy as np

img_fname_full = '/tmp/colmap/images/frame_47_color.jpg'
num_desc = 512
size = 480


img = cv2.cvtColor(cv2.imread(img_fname_full), cv2.COLOR_BGR2RGB)
timg = K.geometry.resize(K.image_to_tensor(img, False).float()/255., size)

/Users/okanarikan/Documents/GitHub/image-matching-benchmark-baselines/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Torch version 1.13.1 has not been tested with coremltools. You may run into unexpected errors. Torch 1.12.1 is the most recent version that has been tested.


In [2]:
keypoint_net = PointModel_v0(is_test=True)
checkpoint = torch.load('third_party/lanet/checkpoints/PointModel_v0.pth', map_location="cpu")
#keypoint_net = PointModel_v1(is_test=True) 
#checkpoint = torch.load('third_party/lanet/checkpoints/PointModel_v1.pth', map_location="cpu")
keypoint_net.load_state_dict(checkpoint['model_state'])
keypoint_net.eval()


PointModel(
  (interestpoint_module): InterestPointModule(
    (conv1): ConvBlock(
      (conv): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (conv2): ConvBlock(
      (conv): Sequential(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [3]:
from extract_lanet import extract_features as lanet_features
kps, resps, descs = lanet_features(img_fname_full, keypoint_net, "cpu", num_desc, size, True)

/Users/okanarikan/Documents/GitHub/image-matching-benchmark-baselines/.venv/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/Users/okanarikan/Documents/GitHub/image-matching-benchmark-baselines/.venv/lib/python3.9/site-packages/torch/nn/functional.py:4227: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


In [4]:


def plot_image(axes, image):
    axes.grid(False)
    axes.imshow(image)
    axes.set_xlim(0, image.shape[1])
    axes.set_ylim(image.shape[0], 0)
    axes.axis('off')

def plot_keypoints(axes, keypoints, color):
    x = [kp[0] for kp in keypoints]
    y = [kp[1] for kp in keypoints]
    axes.scatter(x, y, c=color, marker='.', edgecolors='none')

im = cv2.cvtColor(cv2.imread(img_fname_full), cv2.COLOR_BGR2RGB)


fig = plt.figure(figsize=(25,25))
axes = fig.add_subplot(111)
#k1, k2 = 0, 1
#sm = 12
#cvkp1 = [cv2.KeyPoint(p[0], p[1], sm*s, o) for p, s, o in zip(points[0], scales[0], oris[0])]
#im1_plot = cv2.drawKeypoints(im1, cvkp1[k1:k2], None, None, cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
points = [ kps[i,:] for i in range(kps.shape[0])]
plot_image(axes, im)
plot_keypoints(axes, points, 'yellow')


In [5]:
print(f"Input shape: {timg.shape}")

a, b, c = keypoint_net.forward(timg)
print(f"Out shape: {a.shape}, {b.shape}, {c.shape}")

traced_model = torch.jit.trace(keypoint_net, timg)

a, b, c = traced_model(timg)
print(f"Traced Out shape: {a.shape}, {b.shape}, {c.shape}")

# Input shape is flexible.
input_shape = timg.shape
# input_shape = ct.Shape(shape=(image.shape[0],
#                               image.shape[1],
#                               ct.RangeDim(lower_bound=64, upper_bound=-1, default=image.shape[2]),
#                               ct.RangeDim(lower_bound=64, upper_bound=-1, default=image.shape[3])))

model = ct.convert(traced_model,
                convert_to="mlprogram",
                compute_precision=ct.precision.FLOAT16,
                minimum_deployment_target=ct.target.macOS13,
                # compute_units=ct.ComputeUnit.CPU_AND_NE,
                # compute_units=ct.ComputeUnit.CPU_AND_GPU,
                inputs=[ct.ImageType(name="input", color_layout=ct.colorlayout.GRAYSCALE, scale=1.0/255.0, shape=input_shape)],
                outputs=[ct.TensorType(name="a", dtype=np.float16),
                        ct.TensorType(name="b", dtype=np.float16),
                        ct.TensorType(name="c", dtype=np.float16)])

model.save('/Users/okanarikan/test.mlpackage')

Input shape: torch.Size([1, 3, 480, 640])
Out shape: torch.Size([1, 1, 60, 80]), torch.Size([1, 2, 60, 80]), torch.Size([1, 256, 60, 80])


/Users/okanarikan/Documents/GitHub/image-matching-benchmark-baselines/third_party/lanet/network_v0/modules.py:120: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  coord_norm[:, 0] = (coord_norm[:, 0] / (float(W-1)/2.)) - 1.
/Users/okanarikan/Documents/GitHub/image-matching-benchmark-baselines/third_party/lanet/network_v0/modules.py:121: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  coord_norm[:, 1] = (coord_norm[:, 1] / (float(H-1)/2.)) - 1.
Tuple detected at graph output. This will be flattened in the converted model.


Traced Out shape: torch.Size([1, 1, 60, 80]), torch.Size([1, 2, 60, 80]), torch.Size([1, 256, 60, 80])


Converting PyTorch Frontend ==> MIL Ops:  76%|███████▌  | 213/281 [00:00<00:00, 2805.36 ops/s]


ValueError: In op, of type clip, named 306, the named input `alpha` must have the same data type as the named input `x`. However, alpha has dtype int32 whereas x has dtype fp32.

In [ ]:
from extract_ml_superpoint import extract_features as sp_features
from superpoint import SuperPointFrontend
sp_weights_fname = "third_party/superpoint_ml_repo/superpoint_v1.pth"
superpoint = SuperPointFrontend(sp_weights_fname, 4, 0.00015, 0.7, cuda=False)

In [ ]:
kps, resps, descs = sp_features(img_fname_full, superpoint, "cpu", num_desc, size, True)


fig = plt.figure(figsize=(25,25))
axes = fig.add_subplot(111)
#k1, k2 = 0, 1
#sm = 12
#cvkp1 = [cv2.KeyPoint(p[0], p[1], sm*s, o) for p, s, o in zip(points[0], scales[0], oris[0])]
#im1_plot = cv2.drawKeypoints(im1, cvkp1[k1:k2], None, None, cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
points = [ kps[i,:] for i in range(kps.shape[0])]
plot_image(axes, im)
plot_keypoints(axes, points, 'yellow')
